In [ ]:
import uproot
import pandas as pd
import matplotlib.pyplot as plt
import requests
import zipfile
import io

# Open the ROOT file (update the filename as needed)
file = uproot.open("PhaseSpaceSimulation.root")

# Print the keys in the ROOT file to see what objects it contains:
print("Keys in the ROOT file:", file.keys())

# For example, if the file contains a TTree called "DecayTree", you can access it:
tree = file['PhaseSpaceTree;1']

# List available branches in the tree:
print("Branches in the tree:")
for var in tree.keys():
    print(var)

# Read selected branches into a pandas DataFrame:
# Replace these branch names with correct ones per the corresponding LHCb dataset documentation.
branches_to_load = list(tree.keys())  # adjust as necessary
df = tree.arrays(branches_to_load, library="pd")
print(df.head())

ModuleNotFoundError: No module named 'uproot'

In [3]:
data = df['H1_PX']

# Create a histogram of the data using 30 bins
plt.hist(data, bins=30, color='blue', edgecolor='black', alpha=0.7)

# Label the axes and set the title
plt.xlabel('H1_PX')
plt.ylabel('Frequency')

# Display the plot
plt.show()

NameError: name 'df' is not defined

In [ ]:
def hist(var, range=None):
    plt.hist(df[var], bins=30, color='blue', edgecolor='black', alpha=0.7, range=range)

    # Label the axes and set the title
    plt.xlabel(var)
    plt.ylabel('Frequency')
    
    # Display the plot
    plt.show()

In [ ]:
df['H1_P'] = (df['H1_PX']**2 + df['H1_PY']**2 + df['H1_PZ']**2)**(1/2)

plt.hist(df['H1_P'], bins=30, color='blue', edgecolor='black', alpha=0.7)

# Label the axes and set the title
plt.xlabel('H1 Magnitute of Momentum')
plt.ylabel('Frequency')

# Display the plot
plt.show()

In [ ]:
m_kaon = 493.677 #MeV
df['H1_E'] = (df['H1_P']**2 + m_kaon**2)**(1/2)

plt.hist(df['H1_E'], bins=30, color='blue', edgecolor='black', alpha=0.7)

# Label the axes and set the title
plt.xlabel('H1 Energy')
plt.ylabel('Frequency')

# Display the plot
plt.show()

In [1]:
df['H2_P'] = (df['H2_PX']**2 + df['H2_PY']**2 + df['H2_PZ']**2)**(1/2)
df['H3_P'] = (df['H3_PX']**2 + df['H3_PY']**2 + df['H3_PZ']**2)**(1/2)
df['H2_E'] = (df['H2_P']**2 + m_kaon**2)**(1/2)
df['H3_E'] = (df['H3_P']**2 + m_kaon**2)**(1/2)

NameError: name 'df' is not defined

In [ ]:
df['B_E'] = df['H1_E'] + df['H2_E'] + df['H3_E']
df['B_PX'] = df['H1_PX'] + df['H2_PX'] + df['H3_PX']
df['B_PY'] = df['H1_PY'] + df['H2_PY'] + df['H3_PY']
df['B_PZ'] = df['H1_PZ'] + df['H2_PZ'] + df['H3_PZ']
df['B_P'] = (df['B_PX']**2 + df['B_PY']**2 + df['B_PZ']**2)**(1/2)

df['B_inv_mass'] = (df['B_E']**2 - df['B_P']**2)**(1/2)

hist('B_inv_mass')

In [ ]:
def no_muons_cut(data):
    for index, event in data.iterrows():
        if event['H1_isMuon'] == 1:
            data.drop(index)
        if event['H2_isMuon'] == 1:
            data.drop(index)
        if event['H3_isMuon'] == 1:
            data.drop(index)
    return None

def low_prob_pi_cut(data):
    for index, event in data.iterrows():
        if event['H1_ProbPi'] >= 0.5:
            data.drop(index)
    return None

def high_prob_k_cut(data):
    for index, event in data.iterrows():
        if event['H1_ProbK'] < 0.5:
            data.drop(index)
    return None

In [ ]:
def read_file(path,sample):
    start = time.time() # start the clock
    print("Processing: "+sample) # print which sample is being processed
    data_all = pd.DataFrame() # define empty pandas DataFrame to hold all data for this sample
    tree = uproot.open(path + ":mini")
    numevents = tree.num_entries # number of events
    data_all = pd.concat([data_all, data], ignore_index=True)
    elapsed = time.time() - start # time taken to process
    print("\t nIn: "+str(nIn)+",\t nOut: \t"+str(nOut)+"\t in "+str(round(elapsed,1))+"s") # events before and after
    
    return data_all # return dataframe containing events passing all cuts

In [ ]:
zip_url = 'https://www.dropbox.com/scl/fi/45ewe37xg4q3ekkulq6dk/B2HHH_Data.zip?rlkey=38rbpfbjorc2fvgx0ki777f0c&st=do8et35q&dl=1'
response = requests.get(zip_url)
if response.status_code == 200:
    # Open the zip file from the in-memory bytes
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        # Extract all contents to the current directory (or specify a path)
        zip_ref.extractall(".")
else:
    print("Error downloading file:", response.status_code)

# Now open the ROOT file with uproot
root_file = uproot.open("B2HHH_MagnetUp.root")

In [ ]:
# Open the ROOT file (update the filename as needed)
file1 = uproot.open('B2HHH_MagnetUp.root')
file2 = uproot.open('B2HHH_MagnetDown.root')

# Print the keys in the ROOT file to see what objects it contains:
print("Keys in the ROOT files (1):", file1.keys())
print("Keys in the ROOT files (1):", file2.keys())

In [ ]:
# Applying cuts to the data
print('Data before muon cut:', df.shape)
df1 = no_muons_cut(df)
print('Data after muon cut:', df1.shape)

print('\nData before pi cut:', df1.shape)
df2 = low_prob_pi_cut(df1)
print('Data after pi cut:', df2.shape)

print('\nData before k cut:', df2.shape)
df3 = high_prob_k_cut(df2)
print('Data after k cut:', df3.shape)